In [1]:
%load_ext autoreload
%autoreload 2

import os
import datetime as dt

import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

import torch
import torch.nn as nn


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

import models
import encoders
import decoders
from training import train, test, checkpoint
from dataloaders import load_data, create_loaders

WANDB_API_KEY= '52c84ab3f3b5c1f999c7f5f389f5e423f46fc04a'
import wandb
wandb.login()

cuda


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: raraz15 (use `wandb login --relogin` to force relogin)


True

In [2]:
#dataset_path = '/projects/bassline_transcription/data/datasets'
dataset_path = '/scratch/users/udemir15/ELEC491/bassline_transcription/data/datasets'
dataset_name = 'traxsource_0-5000_bassline_representations'
M = 8 


data_params = {'dataset_path': dataset_path,
               'dataset_name': dataset_name,
               'scale_type': 'min',
               'M': M,
               'batch_size': 128}

X = load_data(data_params)

frequencies = np.unique(X, return_counts=True)

#X = X.reshape(-1, 64)

K = X.max()+1 # Number of classes, assumes consecutive
sequence_length = X.shape[1]

print('Number of classes: {}\nSequence Length: {}'.format(K, sequence_length))
print('Number of data points: {}'.format(X.shape[0]))

train_loader, test_loader = create_loaders(X, data_params)

Number of classes: 38
Sequence Length: 64
Number of data points: 3859


In [25]:
frequencies = np.unique(X, return_counts=True)

In [49]:
cross_entropy_weights = torch.FloatTensor(1 / (frequencies[1] / frequencies[1].max())).to(device)

In [50]:
cross_entropy_weights

tensor([1.9208e+00, 1.9974e+03, 1.2291e+03, 2.8461e+02, 2.4316e+02, 8.1703e+01,
        1.1390e+02, 3.9748e+01, 3.3660e+01, 5.1144e+01, 2.3824e+01, 2.2456e+01,
        6.3654e+00, 2.5358e+01, 2.7591e+01, 2.0903e+01, 5.5018e+01, 3.5307e+01,
        6.4580e+01, 3.2271e+01, 5.1545e+01, 1.0318e+02, 7.1062e+01, 8.7248e+01,
        2.5862e+01, 1.3775e+02, 1.8247e+02, 1.4141e+02, 5.7656e+02, 4.1274e+02,
        7.0792e+02, 5.2513e+02, 1.5115e+03, 3.8570e+03, 1.4526e+03, 3.4954e+03,
        2.0337e+03, 1.0000e+00], device='cuda:0')

import matplotlib.pyplot as plt

fig ,ax = plt.subplots(figsize=(10,5))

ax.bar(*frequencies)

plt.savefig('data_distrib.jpg')
plt.show()

In [42]:
embedding_params = {'num_embeddings': K,
                    'embedding_dim': 32}

encoder_params = {'input_size': embedding_params['embedding_dim'],
                  'hidden_size': embedding_params['embedding_dim'],
                  'dropout': 0.5,  
                  'num_layers': 16,              
                  'batch_size': data_params['batch_size'],
                  'device':device}

#StackedUnidirLSTMDenseDecoder / StackedUnidirectionalLSTMDecoder
decoder_params = {'input_size': encoder_params['hidden_size'],
                  'output_size': K,
                 'num_layers': encoder_params['num_layers'], 
                 'dropout': 0.5,
                 'batch_size': data_params['batch_size'],
                 'sequence_length': sequence_length,
                 'device':device,
                 'teacher_forcing_ratio': 0.5}

#StackedUnidirectionalLSTMDecoderwithEmbedding 
"""decoder_params = {'num_embeddings': K,
                  'embedding_dim': 16,
                  'hidden_size': 128,                  
                  'num_layers': encoder_params['num_layers'], 
                  'dropout': 0.2,
                  'batch_size': data_params['batch_size'],
                  'sequence_length': sequence_length,
                  'device':device}"""

train_params = {'batch_size': data_params['batch_size'],
                'N_epochs': 500}

all_params = {'embedding_params':embedding_params,'encoder_params': encoder_params,
              'decoder_params':decoder_params, 'train_params':train_params}

encoder = encoders.StackedBidirLSTMEncoder(**encoder_params).to(device)

decoder = decoders.StackedUnidirLSTMDecoderwithEmbedding(**decoder_params).to(device)

model = models.AutoEncoder(encoder, decoder, device).to(device)
print(model)
print('Number of parameters: {}'.format(sum([parameter.numel() for parameter in model.parameters()])))

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [51]:
embedding = nn.Embedding(**embedding_params)

encoder = encoders.StackedBidirLSTMEncoder(**encoder_params).to(device)

#decoder = decoders.StackedUnidirLSTMDecoder(**decoder_params).to(device)
decoder = decoders.StackedUnidirLSTMDenseDecoder(**decoder_params).to(device)

model = models.VanillaAutoEncoder(embedding, encoder, decoder).to(device)
print(model)
print('Number of parameters: {}'.format(sum([parameter.numel() for parameter in model.parameters()])))

criterion = nn.CrossEntropyLoss(reduction='mean', weight=cross_entropy_weights)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

VanillaAutoEncoder(
  (embedding): Embedding(38, 32)
  (encoder): StackedBidirLSTMEncoder(
    (net): LSTMnetwork(
      (lstm): LSTM(32, 32, num_layers=16, batch_first=True, dropout=0.5, bidirectional=True)
    )
  )
  (decoder): StackedUnidirLSTMDenseDecoder(
    (rnn): StackedUnidirLSTMDecoder(
      (rnn): LSTMnetwork(
        (lstm): LSTM(32, 32, num_layers=16, batch_first=True, dropout=0.5)
      )
    )
    (dense): Linear(in_features=32, out_features=38, bias=True)
  )
)
Number of parameters: 530854


In [52]:
project_name = 'PyTorch_experiments'

model_name = dt.datetime.strftime(dt.datetime.now(),"%d_%m__%H_%M")

with wandb.init(project=project_name, name=model_name, config=all_params, entity='nbg'):

    test_loss, test_acc = test(model, test_loader, criterion, device)
    print('Test Loss Before Training: {:.6f}'.format(test_loss))
    print('Test Accuracy Before Training: {:.3f}'.format(test_acc))
    wandb.log({'test_loss': test_loss, 'test_accuracy': test_acc})
    
    samples = model.sample(24)
    print('Initial Sample:\n{}'.format(samples[0]))
    wandb.log({'samples': samples})

    for epoch in tqdm(range(train_params['N_epochs'])):

        train_loss, train_acc = train(model, train_loader, optimizer, criterion, device)
        test_loss, test_acc = test(model, test_loader, criterion, device)
        
        model.decoder.update_teacher_forcing_ratio(decoder_params['teacher_forcing_ratio'],
                                                   epoch, train_params['N_epochs']-100)        
        
        wandb.log({'train_loss': train_loss, 'train_accuracy': train_acc,
                  'test_loss': test_loss, 'test_accuracy': test_acc})

        if not (epoch % 25):
            print('Epoch: {}, train_loss: {:.6f}, train_acc: {:.3f}'.format(epoch, train_loss, train_acc))
            checkpoint(model_name, model, optimizer, epoch)
            
        if not (epoch % 50):
            samples = model.sample(24)
            print('Sample:\n{}'.format(samples[0]))
            wandb.log({'samples': samples})
            
    test_loss, test_acc = test(model, test_loader, criterion, device)
    print('Test Loss After Training: {:.6f}'.format(test_loss))
    print('Test Accuracy After Training: {:.3f}'.format(test_acc))
    wandb.log({'test_loss': test_loss, 'test_accuracy': test_acc})
    
    samples = model.sample(24)
    print('Final Sample:\n{}'.format(samples[0]))
    wandb.log({'samples': samples})

  0%|          | 0/500 [00:00<?, ?it/s]

Test Loss Before Training: 3.642147
Test Accuracy Before Training: 0.015
Initial Sample:
tensor([12, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14,
        14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14,
        14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14,
        14, 14, 14, 14, 14, 14, 14, 14, 14, 14], device='cuda:0')
Epoch: 0, train_loss: 3.642235, train_acc: 0.021


  0%|          | 1/500 [00:11<1:33:56, 11.30s/it]

Sample:
tensor([14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14,
        14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14,
        14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14,
        14, 14, 14, 14, 14, 14, 14, 14, 14, 14], device='cuda:0')


  5%|▌         | 26/500 [03:53<1:09:30,  8.80s/it]

...

 80%|████████  | 401/500 [59:04<14:34,  8.84s/it]

Epoch: 400, train_loss: 3.556675, train_acc: 0.025
Sample:
tensor([33,  0,  3, 32, 32,  0, 34,  3, 28, 28, 34, 28, 28,  2,  2, 26, 25,  3,
         3, 28,  0, 34, 34, 28, 28,  0, 28,  2,  2, 26, 26,  3,  3, 28,  0, 13,
        34, 28, 28,  0, 28,  2,  2, 26, 26,  3,  3, 28,  0,  3, 34, 28, 28,  0,
        28,  2,  2, 26, 26,  3,  3, 28,  0,  3], device='cuda:0')


 82%|████████▏ | 410/500 [1:00:24<12:58,  8.65s/it]

wandb: ERROR Control-C detected -- Run data was not synced


KeyboardInterrupt: 

In [ ]:
test_loss, test_acc = test(model, test_loader, criterion, device)

for epoch in tqdm(range(train_params['N_epochs'])):

    train_loss, train_acc = train(model, train_loader, optimizer, criterion, device)
    
    if epoch+1 % 5:
        print('Epoch: {}, train_loss: {:.6f}, acc: {:.3f}'.format(epoch+1, train_loss, train_acc))

test_loss, test_acc = test(model, test_loader, criterion, device)
print('Test Loss After Training: {:.6f}'.format(test_loss))

In [ ]:
T = len(samples[0])
#inp = torch.zeros(K, T)
inp = torch.randn(K, T)

for t, c in enumerate(samples[0]):    
    inp[c,t] = 1
    #inp[c,t] = torch.max(inp)
    
    
    
# 1,K,T
inp = inp.unsqueeze(0).cuda()
output= samples[0].unsqueeze(0).cuda()
print(inp.shape, output.shape)

loss = criterion(inp, output)
print(loss.shape)
print(loss)

encoder_params = {'num_embeddings': K,
                  'embedding_dim': 16, #??
                  'hidden_size': 128,
                  'dropout': 0,  
                  'num_layers': 4,              
                  'batch_size': data_params['batch_size'],
                  'device':device}

decoder_params = {'input_size': encoder_params['hidden_size'],
                  'output_size': K,
                 'num_layers': encoder_params['num_layers'], 
                 'dropout': 0,
                 'batch_size': data_params['batch_size'],
                 'sequence_length': sequence_length,
                 'device':device}

train_params = {'batch_size': data_params['batch_size'],
                'N_epochs': 10}

all_params = {'encoder_params': encoder_params, 'decoder_params':decoder_params, 'train_params':train_params}